## Search queries from Topics

### imports

In [1]:
from elasticsearch import Elasticsearch
from credentials import username, password
from ElasticSettings.topics import create_run, create_eval

### Read information need from xlm file

### Get elasticsearch instance

In [2]:
es = Elasticsearch("http://localhost:9200",
                   basic_auth=(username, password))

In [3]:
topics_path = "data/topics-rnd5_covid-complete.xml"
qrels_path = "data/qrels-covid_d5_j0.5-5_covid-complete.txt"

In [4]:
run_path = create_run(topics_path=topics_path, es=es, index="test_index")
create_eval(qrels_path, run_path=f"data/runs/{run_path}")

In [ ]:
def get_q(search_text: str):
    q = {
        "function_score": {
        "query": {
            "multi_match": {
                    "query": f"{search_text}",
                    "type": "cross_fields",
                    "fields": [
                        "source_x",
                        "title^3",
                        "abstract^2",
                        "journal",
                        "authors"
                ]},
            "functions": [
                {
                    "gauss": {
                    "publish_time": {
                        "origin": datetime.strftime(datetime.now(), "%Y-%m-%d"),
                        "scale": "100Y",
                        "offset": "3Y",
                        "decay": 0.2
                    }
                }}]
            }
        }}
        

    return q

run = "no_operator" + datetime.strftime(datetime.now(), "%d.%m-%H:%M")
with open(f"data/runs/{run}.txt", "w") as f:
    for idx, doc in topics.iterrows():
        q = get_q(doc["query"])
        result = es.search(index="test", body=q, size=1000)
        hits = result["hits"]["hits"]

        for rank, hit in enumerate(hits):
            f.write(" ".join([str(idx), str(0), hit["_source"]["cord_uid"], str(rank), str(hit["_score"]), run, "\n"]))

SyntaxError: '{' was never closed (1076083206.py, line 3)